# Linear regression

In this exercise you will use linear regression to predict flat prices. Training will be handled via gradient descent and we will:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a non-standard loss function.

Let's start with getting the data.

In [1]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2024-10-20 00:57:34--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving www.dropbox.com (www.dropbox.com)... 162.125.72.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.72.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/3x5umw93vtxvmp037wczv/mieszkania.csv?rlkey=dmvzaueu361g7s2w6ui6m9ryb&dl=1 [following]
--2024-10-20 00:57:35--  https://www.dropbox.com/scl/fi/3x5umw93vtxvmp037wczv/mieszkania.csv?rlkey=dmvzaueu361g7s2w6ui6m9ryb&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc9cd5406ddf9b7769dbb8587a8.dl.dropboxusercontent.com/cd/0/inline/CcxE5NroufEmc-p8wU6Mv5i8Nc1KJjDaZ7LyQHslNyqpqiC1j6QB2M7V1KGMJjbhFcuSfZYHMPIooHcJ63EIx1jkcvIngE89lKSiFDYfeSdbBevCsFNdh8oZ5_K666pkF7c/file?dl=1# [following]
--2024-10-20 00:57:35--  https://u

In [2]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`).

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [1]:
from typing import Tuple

import numpy as np
import pandas as pd
from sklearn import preprocessing

def msle(ys, ps):
    assert len(ys) == len(ps)

    n = len(ys)
    result = (1/n) * np.sum((np.log(1 + ys) - np.log(1 + ps))**2)

    return result

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [2]:
# load and prepare the data

def load(name):
    data = pd.read_csv(name)
    xs = data[data.columns[:-1]].to_numpy()
    ys = data[data.columns[-1]].to_numpy()
    return (xs, ys)

x_train, y_train = load('mieszkania.csv')
x_test, y_test = load('mieszkania_test.csv')

def one_hot(arr):
    columns = arr.shape[1]

    for i in range(columns):
        if type(arr[0][i]) != int :

            column = arr[:, i]
            rest = arr[:, [j != i for j in range(columns)]]

            unique = np.unique(column).reshape(1,-1)
            one_hot = np.vstack([column]*len(unique))
            one_hot = (one_hot.T == unique).astype(int)

            return np.hstack([rest, one_hot])

def add_column(value, arr):

    ones_column = np.ones(x_train.shape[0]).reshape(-1,1)
    return np.hstack([ones_column * value, x_train])

x_train, y_train

x_train = one_hot(x_train)

x_train = add_column(1, x_train).astype(float)

y_train = (y_train / np.max(y_train)).reshape(-1,1).astype(float)

In [3]:
###################################################
# TODO: Compute msle for outputing the mean price #
###################################################
def msle(xs, ps):
    assert len(xs) == len(ps)

    n = len(xs)
    sum = np.sum((np.log(1 + xs) - np.log(1 + ps)) ** 2)

    return (1/n) * sum

mean_price = np.mean(y_train)

msle(y_train, np.ones(x_train.shape[0]) * mean_price)

np.float64(5.468949827661844)

Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [4]:
#############################################
# TODO: Find this constant and compute msle #
#############################################

# (log(1 + p) - log(1 + x)) ** 2

msle_mean = (np.e ** ((1 / len(y_train)) * np.sum(np.log(1 + y_train)))) - 1

msle(y_train, np.ones(x_train.shape[0]) * msle_mean)

np.float64(5.431039782425594)

Now, let's implement a standard linear regression model.

In [40]:
##########################################################
# TODO: Implement linear regression and compute its msle #
##########################################################

def msle(xs, ps):
    assert len(xs) == len(ps)

    n = len(xs)
    sum = np.sum((np.log(1 + xs) - np.log(1 + ps)) ** 2)

    return (1/n) * sum

def mse(xs, ps):
    assert len(xs) == len(ps)

    n = len(xs)
    sum = np.sum((xs - ps) ** 2)

    return (1/n) * sum


In [ ]:
n, features = x_train.shape
w = np.zeros(features).reshape(-1, 1)
lr = 0.5 # step size
n_epochs = 3

print(y_train.shape)
def predict(w, xs):
    #print(xs.shape, w.shape, xs, "\n", w, "\n", xs @ w, "\n","\n")
    
    return xs @ w

def evaluate(w, xs, ys):
    return mse(ys, predict(w, xs))

losses = [evaluate(w, x_train, y_train)]

for i in range(n_epochs):
    dJdwi = (2/n) * np.sum((predict(w, x_train) - y_train) * x_train, axis = 0)

    w = w - lr * dJdwi.reshape(-1, 1)

    loss = evaluate(w, x_train, y_train)
    losses.append(loss)

    print(f'Iter: {i:>3} Loss: {loss:8.8f} w: {w.T}')

(200, 1)


TypeError: unsupported format string passed to numpy.ndarray.__format__

Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint:
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>

In [ ]:
#############################################
# TODO: Optimize msle and compare the error #
#############################################

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code.

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint:
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2:
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [ ]:
###############################################
# TODO: Implement the feature engineering part #
###############################################

In [ ]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################